# v1

elo 개론소개 

- 최초 경기를 하는 사람은 1500점이 부여됨 
- 이기면 점수가 증가, 질 경우 점수 감소 
- 점수차가 큰 사람을 이길 경우 증가하는 점수의 폭이 크고, 점수차가 작은 사람을 이길 경우, 증가하는 점수의 폭이 작다 

결론적으로 상대적인 승률을 가지고, 경기별로 조정한다. 

In [6]:
import pandas as pd

# Define the file path
file_path = './data/ku_master_sheet.xlsx'

# Load the data from the 'players' and 'matches' sheets
players_df = pd.read_excel(file_path, sheet_name='players', skiprows=2, engine='openpyxl')

# Drop any empty columns if necessary
players_df = players_df.dropna(how='all', axis=1)

# Load the data from the 'matches' sheet, selecting columns A to K
matches_df = pd.read_excel(file_path, sheet_name='matches', usecols='A:K', engine='openpyxl')

matches_df = matches_df.dropna()

# Display the loaded data
print("Players DataFrame:")
display(players_df.head())

print("\nMatches DataFrame:")
display(matches_df.tail())

Players DataFrame:


,닉네임,성명,성별,시작일자,비고
0,성현,임성현,남,2208,NaN
1,예스욱,김정욱,남,2208,NaN
2,쥬드,현종현,남,2102,NaN
3,김연주,김연주,여,2201,NaN
4,미지,신미진,여,2102,NaN



Matches DataFrame:


,date,match_num,team1_forehand,team1_backhand,team2_forehand,team2_backhand,team1_score,team_2_score,match_type,result,game_type
58,2024-10-27,4,쿤,김연주,태보,미지,4,2,혼복,team1,4게임
59,2024-10-27,5,쿤,정현,쥬드,태보,4,0,남복,team1,4게임
60,2024-10-27,6,김연주,시호,파란하늘,미지,4,3,여복,team1,4게임
61,2024-10-27,7,쥬드,민수,정현,태보,4,3,남복,team1,4게임
62,2024-10-27,8,김연주,예스욱,미지,태보,1,5,혼복,team2,6게임


In [20]:
players_df

,닉네임,성명,성별,시작일자,비고
0,성현,임성현,남,2208,NaN
1,예스욱,김정욱,남,2208,NaN
2,쥬드,현종현,남,2102,NaN
3,김연주,김연주,여,2201,NaN
4,미지,신미진,여,2102,NaN
5,태보,심태보,남,2207,NaN
6,정현,한정현,남,2212,NaN
7,파란하늘,파란하늘,여,2208,NaN
8,챨리심,찰리심,남,2205,NaN
9,kohei,kohei tada,남,?,NaN


In [19]:
import pandas as pd

# Initialize ELO ratings for all players
elo_ratings = {player: 1500 for player in players_df['닉네임'].values}

# Set K-factor for ELO calculations (commonly 32)
K = 32

def expected_score(team1_rating, team2_rating):
    """Calculate the expected score for team1 against team2."""
    return 1 / (1 + 10 ** ((team2_rating - team1_rating) / 400))

def update_elo(team1_rating, team2_rating, team1_score, team2_score):
    """Update the ELO ratings for both teams."""
    expected_team1 = expected_score(team1_rating, team2_rating)
    expected_team2 = 1 - expected_team1
    
    # Determine the actual score based on match result
    if team1_score > team2_score:
        actual_team1 = 1
        actual_team2 = 0
    elif team1_score < team2_score:
        actual_team1 = 0
        actual_team2 = 1
    else:
        actual_team1 = 0.5
        actual_team2 = 0.5
    
    # Update ratings
    new_team1_rating = team1_rating + K * (actual_team1 - expected_team1)
    new_team2_rating = team2_rating + K * (actual_team2 - expected_team2)
    
    return new_team1_rating, new_team2_rating

# Iterate over matches in date and match_num order
for index, row in matches_df.sort_values(['date', 'match_num']).iterrows():
    # Get player ratings for team1 and team2 (average for doubles)
    team1_players = [row['team1_forehand'], row['team1_backhand']]
    team2_players = [row['team2_forehand'], row['team2_backhand']]
    
    team1_rating = (elo_ratings[team1_players[0]] + elo_ratings[team1_players[1]]) / 2
    team2_rating = (elo_ratings[team2_players[0]] + elo_ratings[team2_players[1]]) / 2
    
    # Update ratings based on the match result
    new_team1_rating, new_team2_rating = update_elo(
        team1_rating, team2_rating, row['team1_score'], row['team_2_score']
    )
    
    # Distribute the new team ratings back to individual players
    for player in team1_players:
        elo_ratings[player] += (new_team1_rating - team1_rating) / 2
    for player in team2_players:
        elo_ratings[player] += (new_team2_rating - team2_rating) / 2

# Display the final ELO ratings
elo_df = pd.DataFrame(list(elo_ratings.items()), columns=['Player', 'ELO'])
display(elo_df.sort_values(by='ELO',ascending=False))


,Player,ELO
1,예스욱,1564.487896
2,쥬드,1544.399444
4,미지,1525.952301
14,민수,1524.868635
11,쿤,1524.791244
17,시호,1508.528135
9,kohei,1506.299345
10,유하,1505.573989
15,혜빈,1499.754290
12,태규,1499.108540


In [32]:
import pandas as pd

# Initialize ELO ratings for all players based on gender
elo_ratings = {row['닉네임']: 1500 if row['성별'] == '남' else 1450 for _, row in players_df.iterrows()}

# Set K-factor for ELO calculations (commonly 32)
K = 32

def expected_score(team1_rating, team2_rating):
    """Calculate the expected score for team1 against team2."""
    return 1 / (1 + 10 ** ((team2_rating - team1_rating) / 400))

def update_elo(team1_rating, team2_rating, team1_score, team2_score):
    """Update the ELO ratings for both teams."""
    expected_team1 = expected_score(team1_rating, team2_rating)
    expected_team2 = 1 - expected_team1
    
    # Determine the actual score based on match result
    if team1_score > team2_score:
        actual_team1 = 1
        actual_team2 = 0
    elif team1_score < team2_score:
        actual_team1 = 0
        actual_team2 = 1
    else:
        actual_team1 = 0.5
        actual_team2 = 0.5
    
    # Update ratings
    new_team1_rating = team1_rating + K * (actual_team1 - expected_team1)
    new_team2_rating = team2_rating + K * (actual_team2 - expected_team2)
    
    return new_team1_rating, new_team2_rating

# Iterate over matches in date and match_num order
for index, row in matches_df.sort_values(['date', 'match_num']).iterrows():
    # Get player ratings for team1 and team2 (average for doubles)
    team1_players = [row['team1_forehand'], row['team1_backhand']]
    team2_players = [row['team2_forehand'], row['team2_backhand']]
    
    team1_rating = (elo_ratings[team1_players[0]] + elo_ratings[team1_players[1]]) / 2
    team2_rating = (elo_ratings[team2_players[0]] + elo_ratings[team2_players[1]]) / 2
    
    # Update ratings based on the match result
    new_team1_rating, new_team2_rating = update_elo(
        team1_rating, team2_rating, row['team1_score'], row['team_2_score']
    )
    
    # Distribute the new team ratings back to individual players
    for player in team1_players:
        elo_ratings[player] += (new_team1_rating - team1_rating) / 2
    for player in team2_players:
        elo_ratings[player] += (new_team2_rating - team2_rating) / 2

# Add gender to the final ELO DataFrame
elo_df = pd.DataFrame([
    {'Player': player, 'ELO': rating, 'Gender': players_df.loc[players_df['닉네임'] == player, '성별'].values[0]}
    for player, rating in elo_ratings.items()
])


# display(elo_df.sort_values(by='ELO', ascending=False).reset_index(drop=True))

# Display ELO ratings grouped by gender and sorted in descending order within each group
elo_grouped_df = (
    elo_df.sort_values(by=['Gender', 'ELO'], ascending=[True, False])
    .groupby('Gender')
    .apply(lambda x: x.reset_index(drop=True))
)

display(elo_grouped_df)


Player          ELO Gender
Gender                              
남      0     예스욱  1564.888096      남
       1      쥬드  1544.572691      남
       2      민수  1524.894221      남
       3       쿤  1524.772523      남
       4   kohei  1505.720122      남
       5      태규  1499.039393      남
       6      정현  1493.683985      남
       7      똥시  1491.352055      남
       8     챨리심  1460.982142      남
       9      성현  1460.955254      남
       10     태보  1441.885251      남
여      0      미지  1477.574333      여
       1      시호  1458.569260      여
       2      유하  1455.707399      여
       3      혜빈  1449.760296      여
       4    파란하늘  1437.943336      여
       5    테니스율  1434.414732      여
       6     김연주  1423.284910      여

In [26]:
import pandas as pd

# Sample players and matches data
players = players_df['닉네임'].values  # Assuming players_df has been loaded
matches = matches_df  # Assuming matches_df has been loaded

# Initialize a DataFrame for pairwise results with empty strings
pairwise_results = pd.DataFrame('', index=players, columns=players)

# Fill the DataFrame with results
for index, row in matches.iterrows():
    team1 = [row['team1_forehand'], row['team1_backhand']]
    team2 = [row['team2_forehand'], row['team2_backhand']]
    
    # Determine match result
    if row['team1_score'] > row['team_2_score']:
        result_team1 = 'W'  # Team 1 wins
        result_team2 = 'L'  # Team 2 loses
    elif row['team1_score'] < row['team_2_score']:
        result_team1 = 'L'  # Team 1 loses
        result_team2 = 'W'  # Team 2 wins
    else:
        result_team1 = 'D'  # Draw
        result_team2 = 'D'  # Draw
    
    # Update pairwise results for team1
    for player1 in team1:
        for player2 in team1:
            if player1 != player2:
                pairwise_results.at[player1, player2] += (result_team1 + ', ') if pairwise_results.at[player1, player2] else result_team1

    # Update pairwise results for team2
    for player1 in team2:
        for player2 in team2:
            if player1 != player2:
                pairwise_results.at[player1, player2] += (result_team2 + ', ') if pairwise_results.at[player1, player2] else result_team2

    # Update pairwise results for opponents
    for player1 in team1:
        for player2 in team2:
            if result_team1 == 'W':
                pairwise_results.at[player1, player2] += 'W, ' if pairwise_results.at[player1, player2] else 'W'
                pairwise_results.at[player2, player1] += 'L, ' if pairwise_results.at[player2, player1] else 'L'
            elif result_team1 == 'L':
                pairwise_results.at[player1, player2] += 'L, ' if pairwise_results.at[player1, player2] else 'L'
                pairwise_results.at[player2, player1] += 'W, ' if pairwise_results.at[player2, player1] else 'W'
            else:  # Draw
                pairwise_results.at[player1, player2] += 'D, ' if pairwise_results.at[player1, player2] else 'D'
                pairwise_results.at[player2, player1] += 'D, ' if pairwise_results.at[player2, player1] else 'D'

# Clean up the DataFrame by removing trailing commas and spaces
pairwise_results = pairwise_results.applymap(lambda x: x[:-2] if isinstance(x, str) and x.endswith(', ') else x)

# Display the pairwise results DataFrame clearly
pairwise_results = pairwise_results.fillna('')  # Replace NaN with empty strings for better display
pairwise_results.index.name = 'Player'
pairwise_results.columns.name = 'Opponent'

# Display the pairwise results
print(pairwise_results)


Opponent                    성현                      예스욱  \
Player                                                    
성현                                 DL, L, L, W, L, L, L   
예스욱       DW, L, W, W, W, W, W                            
쥬드           WW, W, W, W, W, L        LL, L, W, W, L, W   
김연주             DL, W, D, L, W                       DL   
미지                          WL                       WW   
태보                 LL, W, L, W        WL, W, W, D, L, W   
정현                 DL, W, L, W  DL, L, W, L, D, L, D, W   
파란하늘            LW, W, W, L, L                    LL, L   
챨리심                      LL, L                        L   
kohei                                                     
유하              WL, D, L, L, W                       WW   
쿤                           WD                       WL   
태규                          WL                       DL   
똥시                                                        
민수                           L              DL, D, W, L 

In [27]:
import pandas as pd

# Assuming players_df and matches_df have already been loaded with your actual data
# For demonstration purposes, let's assume some sample data
# players_df = pd.read_excel('path_to_your_players_sheet.xlsx')
# matches_df = pd.read_excel('path_to_your_matches_sheet.xlsx')

# Get player nicknames from players DataFrame
players = players_df['닉네임'].values  # Get player nicknames from players DataFrame
matches = matches_df  # Load matches DataFrame

# Initialize DataFrames for pairwise results
same_team_results = pd.DataFrame(columns=players, index=players).fillna('0전 0승 0패 0무')
opponent_results = pd.DataFrame(columns=players, index=players).fillna('0전 0승 0패 0무')

# Function to update results
def update_results(df, player1, player2, result):
    current = df.at[player1, player2]
    current_matches, current_wins, current_losses, current_draws = map(int, [
        current.split('전')[0],
        current.split(' ')[1].replace('승', ''),
        current.split(' ')[2].replace('패', ''),
        current.split(' ')[3].replace('무', '')
    ])
    
    current_matches += 1  # Increment the match count
    if result == 'win':
        current_wins += 1
    elif result == 'loss':
        current_losses += 1
    else:  # result == 'draw'
        current_draws += 1
    
    df.at[player1, player2] = f"{current_matches}전 {current_wins}승 {current_losses}패 {current_draws}무"

# Fill the DataFrames with results
for index, row in matches.iterrows():
    team1 = [row['team1_forehand'], row['team1_backhand']]
    team2 = [row['team2_forehand'], row['team2_backhand']]
    
    # Determine match result
    if row['team1_score'] > row['team_2_score']:
        result_team1 = 'win'
        result_team2 = 'loss'
    elif row['team1_score'] < row['team_2_score']:
        result_team1 = 'loss'
        result_team2 = 'win'
    else:
        result_team1 = 'draw'
        result_team2 = 'draw'

    # Update same team results
    for player1 in team1:
        for player2 in team1:
            if player1 != player2:
                update_results(same_team_results, player1, player2, result_team1)
                
    for player1 in team2:
        for player2 in team2:
            if player1 != player2:
                update_results(same_team_results, player1, player2, result_team2)

    # Update opponent results
    for player1 in team1:
        for player2 in team2:
            update_results(opponent_results, player1, player2, result_team1)
            update_results(opponent_results, player2, player1, result_team2)

# Clean the outputs to avoid multiple entries for the same match
same_team_results = same_team_results.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
opponent_results = opponent_results.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Set index and column names for clarity
same_team_results.index.name = 'Player'
same_team_results.columns.name = 'Teammate'

opponent_results.index.name = 'Player'
opponent_results.columns.name = 'Opponent'

# Display the results at the end
print("Same Team Results:")
display(same_team_results[:11])

print("\nOpponent Results:")
display(opponent_results[:11])

Same Team Results:


Teammate,성현,예스욱,쥬드,김연주,미지,태보,정현,파란하늘,챨리심,kohei,유하,쿤,태규,똥시,민수,혜빈,테니스율,시호
Player,,,,,,,,,,,,,,,,,,
성현,0전 0승 0패 0무,2전 1승 1패 0무,2전 2승 0패 0무,3전 1승 2패 0무,1전 1승 0패 0무,1전 0승 1패 0무,3전 1승 1패 1무,1전 0승 1패 0무,1전 0승 1패 0무,0전 0승 0패 0무,3전 0승 2패 1무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,1전 0승 1패 0무,1전 0승 1패 0무,0전 0승 0패 0무
예스욱,2전 1승 1패 0무,0전 0승 0패 0무,2전 2승 0패 0무,2전 0승 1패 1무,1전 1승 0패 0무,3전 2승 1패 0무,2전 1승 0패 1무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,2전 2승 0패 0무,1전 1승 0패 0무,1전 0승 0패 1무,0전 0승 0패 0무,0전 0승 0패 0무,1전 1승 0패 0무,1전 1승 0패 0무,0전 0승 0패 0무
쥬드,2전 2승 0패 0무,2전 2승 0패 0무,0전 0승 0패 0무,3전 2승 1패 0무,3전 3승 0패 0무,4전 1승 3패 0무,2전 0승 2패 0무,4전 1승 3패 0무,1전 1승 0패 0무,2전 2승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,1전 1승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무
김연주,3전 1승 2패 0무,2전 0승 1패 1무,3전 2승 1패 0무,0전 0승 0패 0무,1전 0승 1패 0무,2전 0승 2패 0무,2전 0승 1패 1무,1전 1승 0패 0무,1전 0승 1패 0무,2전 0승 2패 0무,1전 1승 0패 0무,2전 1승 0패 1무,0전 0승 0패 0무,1전 0승 1패 0무,1전 1승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,1전 1승 0패 0무
미지,1전 1승 0패 0무,1전 1승 0패 0무,3전 3승 0패 0무,1전 0승 1패 0무,0전 0승 0패 0무,3전 1승 2패 0무,3전 2승 1패 0무,3전 1승 2패 0무,2전 2승 0패 0무,0전 0승 0패 0무,1전 0승 1패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무
태보,1전 0승 1패 0무,3전 2승 1패 0무,4전 1승 3패 0무,2전 0승 2패 0무,3전 1승 2패 0무,0전 0승 0패 0무,4전 1승 3패 0무,2전 0승 2패 0무,1전 0승 1패 0무,1전 1승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,2전 1승 0패 1무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무
정현,3전 1승 1패 1무,2전 1승 0패 1무,2전 0승 2패 0무,2전 0승 1패 1무,3전 2승 1패 0무,4전 1승 3패 0무,0전 0승 0패 0무,9전 4승 4패 1무,2전 1승 1패 0무,0전 0승 0패 0무,2전 1승 1패 0무,1전 1승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,2전 1승 0패 1무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무
파란하늘,1전 0승 1패 0무,0전 0승 0패 0무,4전 1승 3패 0무,1전 1승 0패 0무,3전 1승 2패 0무,2전 0승 2패 0무,9전 4승 4패 1무,0전 0승 0패 0무,3전 0승 2패 1무,0전 0승 0패 0무,2전 2승 0패 0무,0전 0승 0패 0무,1전 1승 0패 0무,1전 1승 0패 0무,0전 0승 0패 0무,1전 1승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무
챨리심,1전 0승 1패 0무,0전 0승 0패 0무,1전 1승 0패 0무,1전 0승 1패 0무,2전 2승 0패 0무,1전 0승 1패 0무,2전 1승 1패 0무,3전 0승 2패 1무,0전 0승 0패 0무,0전 0승 0패 0무,2전 0승 2패 0무,0전 0승 0패 0무,0전 0승 0패 0무,1전 0승 1패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무



Opponent Results:


Opponent,성현,예스욱,쥬드,김연주,미지,태보,정현,파란하늘,챨리심,kohei,유하,쿤,태규,똥시,민수,혜빈,테니스율,시호
Player,,,,,,,,,,,,,,,,,,
성현,0전 0승 0패 0무,6전 0승 5패 1무,5전 1승 4패 0무,3전 0승 1패 2무,1전 1승 0패 0무,4전 2승 2패 0무,2전 1승 1패 0무,5전 2승 3패 0무,2전 2승 0패 0무,0전 0승 0패 0무,3전 1승 2패 0무,2전 0승 1패 1무,2전 1승 1패 0무,0전 0승 0패 0무,1전 1승 0패 0무,1전 0승 1패 0무,1전 0승 1패 0무,0전 0승 0패 0무
예스욱,6전 5승 0패 1무,0전 0승 0패 0무,5전 4승 1패 0무,0전 0승 0패 0무,1전 0승 1패 0무,4전 1승 2패 1무,7전 4승 1패 2무,3전 3승 0패 0무,1전 1승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,1전 1승 0패 0무,1전 1승 0패 0무,0전 0승 0패 0무,5전 2승 1패 2무,1전 1승 0패 0무,1전 1승 0패 0무,0전 0승 0패 0무
쥬드,5전 4승 1패 0무,5전 1승 4패 0무,0전 0승 0패 0무,0전 0승 0패 0무,4전 1승 3패 0무,7전 6승 1패 0무,13전 7승 6패 0무,2전 2승 0패 0무,5전 4승 1패 0무,0전 0승 0패 0무,3전 3승 0패 0무,3전 1승 2패 0무,0전 0승 0패 0무,0전 0승 0패 0무,1전 1승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무
김연주,3전 1승 0패 2무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,8전 4승 4패 0무,3전 1승 2패 0무,8전 1승 6패 1무,10전 3승 6패 1무,4전 2승 1패 1무,0전 0승 0패 0무,8전 4승 3패 1무,1전 0승 1패 0무,0전 0승 0패 0무,1전 0승 1패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무
미지,1전 0승 1패 0무,1전 1승 0패 0무,4전 3승 1패 0무,8전 4승 4패 0무,0전 0승 0패 0무,2전 2승 0패 0무,5전 4승 1패 0무,8전 5승 3패 0무,1전 1승 0패 0무,1전 1승 0패 0무,3전 1승 2패 0무,1전 0승 1패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,1전 0승 1패 0무
태보,4전 2승 2패 0무,4전 2승 1패 1무,7전 1승 6패 0무,3전 2승 1패 0무,2전 0승 2패 0무,0전 0승 0패 0무,8전 1승 6패 1무,4전 1승 3패 0무,5전 3승 2패 0무,2전 1승 1패 0무,1전 0승 1패 0무,2전 0승 2패 0무,0전 0승 0패 0무,2전 1승 1패 0무,2전 0승 2패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무
정현,2전 1승 1패 0무,7전 1승 4패 2무,13전 6승 7패 0무,8전 6승 1패 1무,5전 1승 4패 0무,8전 6승 1패 1무,0전 0승 0패 0무,4전 2승 1패 1무,4전 1승 2패 1무,2전 1승 1패 0무,3전 0승 2패 1무,1전 0승 1패 0무,2전 0승 0패 2무,1전 1승 0패 0무,4전 0승 3패 1무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무
파란하늘,5전 3승 2패 0무,3전 0승 3패 0무,2전 0승 2패 0무,10전 6승 3패 1무,8전 3승 5패 0무,4전 3승 1패 0무,4전 1승 2패 1무,0전 0승 0패 0무,2전 1승 1패 0무,2전 0승 2패 0무,7전 3승 3패 1무,0전 0승 0패 0무,1전 0승 0패 1무,1전 1승 0패 0무,2전 0승 2패 0무,1전 1승 0패 0무,3전 2승 1패 0무,1전 0승 1패 0무
챨리심,2전 0승 2패 0무,1전 0승 1패 0무,5전 1승 4패 0무,4전 1승 2패 1무,1전 0승 1패 0무,5전 2승 3패 0무,4전 2승 1패 1무,2전 1승 1패 0무,0전 0승 0패 0무,3전 1승 2패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,1전 0승 1패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무


In [28]:
import pandas as pd

# Assuming players_df and matches_df have already been loaded with your actual data
# Get player nicknames from players DataFrame
players = players_df['닉네임'].values  # Get player nicknames from players DataFrame
matches = matches_df  # Load matches DataFrame

# Initialize DataFrames for pairwise results
same_team_results = pd.DataFrame(columns=players, index=players)
opponent_results = pd.DataFrame(columns=players, index=players)

# Fill with default values and leave self-matches blank
for player in players:
    for other_player in players:
        if player == other_player:
            same_team_results.at[player, other_player] = ''  # Leave empty for self-match
            opponent_results.at[player, other_player] = ''  # Leave empty for self-match
        else:
            same_team_results.at[player, other_player] = '0전 0승 0패 0무'
            opponent_results.at[player, other_player] = '0전 0승 0패 0무'

# Function to update results
def update_results(df, player1, player2, result):
    current = df.at[player1, player2]
    current_matches, current_wins, current_losses, current_draws = map(int, [
        current.split('전')[0],
        current.split(' ')[1].replace('승', ''),
        current.split(' ')[2].replace('패', ''),
        current.split(' ')[3].replace('무', '')
    ])
    
    current_matches += 1  # Increment the match count
    if result == 'win':
        current_wins += 1
    elif result == 'loss':
        current_losses += 1
    else:  # result == 'draw'
        current_draws += 1
    
    df.at[player1, player2] = f"{current_matches}전 {current_wins}승 {current_losses}패 {current_draws}무"

# Fill the DataFrames with results
for index, row in matches.iterrows():
    team1 = [row['team1_forehand'], row['team1_backhand']]
    team2 = [row['team2_forehand'], row['team2_backhand']]
    
    # Determine match result
    if row['team1_score'] > row['team_2_score']:
        result_team1 = 'win'
        result_team2 = 'loss'
    elif row['team1_score'] < row['team_2_score']:
        result_team1 = 'loss'
        result_team2 = 'win'
    else:
        result_team1 = 'draw'
        result_team2 = 'draw'

    # Update same team results
    for player1 in team1:
        for player2 in team1:
            if player1 != player2:
                update_results(same_team_results, player1, player2, result_team1)
                
    for player1 in team2:
        for player2 in team2:
            if player1 != player2:
                update_results(same_team_results, player1, player2, result_team2)

    # Update opponent results
    for player1 in team1:
        for player2 in team2:
            if player1 != player2:  # Only update if players are not the same
                update_results(opponent_results, player1, player2, result_team1)
                update_results(opponent_results, player2, player1, result_team2)

# Clean the outputs to avoid multiple entries for the same match
same_team_results = same_team_results.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
opponent_results = opponent_results.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Set index and column names for clarity
same_team_results.index.name = 'Player'
same_team_results.columns.name = 'Teammate'

opponent_results.index.name = 'Player'
opponent_results.columns.name = 'Opponent'

# Display the first 11 rows and columns of the results
print("Same Team Results:")
display(same_team_results.iloc[:11, :11])  # Display first 11 rows and columns

print("\nOpponent Results:")
display(opponent_results.iloc[:11, :11])  # Display first 11 rows and columns


# Display the first 11 rows and columns of the results
print("Same Team Results:")
display(same_team_results.iloc[:, :])  # Display first 11 rows and columns

print("\nOpponent Results:")
display(opponent_results.iloc[:, :])  # Display first 11 rows and columns


Same Team Results:


Teammate,성현,예스욱,쥬드,김연주,미지,태보,정현,파란하늘,챨리심,kohei,유하
Player,,,,,,,,,,,
성현,,2전 1승 1패 0무,2전 2승 0패 0무,3전 1승 2패 0무,1전 1승 0패 0무,1전 0승 1패 0무,3전 1승 1패 1무,1전 0승 1패 0무,1전 0승 1패 0무,0전 0승 0패 0무,3전 0승 2패 1무
예스욱,2전 1승 1패 0무,,2전 2승 0패 0무,2전 0승 1패 1무,1전 1승 0패 0무,3전 2승 1패 0무,2전 1승 0패 1무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,2전 2승 0패 0무
쥬드,2전 2승 0패 0무,2전 2승 0패 0무,,3전 2승 1패 0무,3전 3승 0패 0무,4전 1승 3패 0무,2전 0승 2패 0무,4전 1승 3패 0무,1전 1승 0패 0무,2전 2승 0패 0무,0전 0승 0패 0무
김연주,3전 1승 2패 0무,2전 0승 1패 1무,3전 2승 1패 0무,,1전 0승 1패 0무,2전 0승 2패 0무,2전 0승 1패 1무,1전 1승 0패 0무,1전 0승 1패 0무,2전 0승 2패 0무,1전 1승 0패 0무
미지,1전 1승 0패 0무,1전 1승 0패 0무,3전 3승 0패 0무,1전 0승 1패 0무,,3전 1승 2패 0무,3전 2승 1패 0무,3전 1승 2패 0무,2전 2승 0패 0무,0전 0승 0패 0무,1전 0승 1패 0무
태보,1전 0승 1패 0무,3전 2승 1패 0무,4전 1승 3패 0무,2전 0승 2패 0무,3전 1승 2패 0무,,4전 1승 3패 0무,2전 0승 2패 0무,1전 0승 1패 0무,1전 1승 0패 0무,0전 0승 0패 0무
정현,3전 1승 1패 1무,2전 1승 0패 1무,2전 0승 2패 0무,2전 0승 1패 1무,3전 2승 1패 0무,4전 1승 3패 0무,,9전 4승 4패 1무,2전 1승 1패 0무,0전 0승 0패 0무,2전 1승 1패 0무
파란하늘,1전 0승 1패 0무,0전 0승 0패 0무,4전 1승 3패 0무,1전 1승 0패 0무,3전 1승 2패 0무,2전 0승 2패 0무,9전 4승 4패 1무,,3전 0승 2패 1무,0전 0승 0패 0무,2전 2승 0패 0무
챨리심,1전 0승 1패 0무,0전 0승 0패 0무,1전 1승 0패 0무,1전 0승 1패 0무,2전 2승 0패 0무,1전 0승 1패 0무,2전 1승 1패 0무,3전 0승 2패 1무,,0전 0승 0패 0무,2전 0승 2패 0무



Opponent Results:


Opponent,성현,예스욱,쥬드,김연주,미지,태보,정현,파란하늘,챨리심,kohei,유하
Player,,,,,,,,,,,
성현,,6전 0승 5패 1무,5전 1승 4패 0무,3전 0승 1패 2무,1전 1승 0패 0무,4전 2승 2패 0무,2전 1승 1패 0무,5전 2승 3패 0무,2전 2승 0패 0무,0전 0승 0패 0무,3전 1승 2패 0무
예스욱,6전 5승 0패 1무,,5전 4승 1패 0무,0전 0승 0패 0무,1전 0승 1패 0무,4전 1승 2패 1무,7전 4승 1패 2무,3전 3승 0패 0무,1전 1승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무
쥬드,5전 4승 1패 0무,5전 1승 4패 0무,,0전 0승 0패 0무,4전 1승 3패 0무,7전 6승 1패 0무,13전 7승 6패 0무,2전 2승 0패 0무,5전 4승 1패 0무,0전 0승 0패 0무,3전 3승 0패 0무
김연주,3전 1승 0패 2무,0전 0승 0패 0무,0전 0승 0패 0무,,8전 4승 4패 0무,3전 1승 2패 0무,8전 1승 6패 1무,10전 3승 6패 1무,4전 2승 1패 1무,0전 0승 0패 0무,8전 4승 3패 1무
미지,1전 0승 1패 0무,1전 1승 0패 0무,4전 3승 1패 0무,8전 4승 4패 0무,,2전 2승 0패 0무,5전 4승 1패 0무,8전 5승 3패 0무,1전 1승 0패 0무,1전 1승 0패 0무,3전 1승 2패 0무
태보,4전 2승 2패 0무,4전 2승 1패 1무,7전 1승 6패 0무,3전 2승 1패 0무,2전 0승 2패 0무,,8전 1승 6패 1무,4전 1승 3패 0무,5전 3승 2패 0무,2전 1승 1패 0무,1전 0승 1패 0무
정현,2전 1승 1패 0무,7전 1승 4패 2무,13전 6승 7패 0무,8전 6승 1패 1무,5전 1승 4패 0무,8전 6승 1패 1무,,4전 2승 1패 1무,4전 1승 2패 1무,2전 1승 1패 0무,3전 0승 2패 1무
파란하늘,5전 3승 2패 0무,3전 0승 3패 0무,2전 0승 2패 0무,10전 6승 3패 1무,8전 3승 5패 0무,4전 3승 1패 0무,4전 1승 2패 1무,,2전 1승 1패 0무,2전 0승 2패 0무,7전 3승 3패 1무
챨리심,2전 0승 2패 0무,1전 0승 1패 0무,5전 1승 4패 0무,4전 1승 2패 1무,1전 0승 1패 0무,5전 2승 3패 0무,4전 2승 1패 1무,2전 1승 1패 0무,,3전 1승 2패 0무,0전 0승 0패 0무


Same Team Results:


Teammate,성현,예스욱,쥬드,김연주,미지,태보,정현,파란하늘,챨리심,kohei,유하,쿤,태규,똥시,민수,혜빈,테니스율,시호
Player,,,,,,,,,,,,,,,,,,
성현,,2전 1승 1패 0무,2전 2승 0패 0무,3전 1승 2패 0무,1전 1승 0패 0무,1전 0승 1패 0무,3전 1승 1패 1무,1전 0승 1패 0무,1전 0승 1패 0무,0전 0승 0패 0무,3전 0승 2패 1무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,1전 0승 1패 0무,1전 0승 1패 0무,0전 0승 0패 0무
예스욱,2전 1승 1패 0무,,2전 2승 0패 0무,2전 0승 1패 1무,1전 1승 0패 0무,3전 2승 1패 0무,2전 1승 0패 1무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,2전 2승 0패 0무,1전 1승 0패 0무,1전 0승 0패 1무,0전 0승 0패 0무,0전 0승 0패 0무,1전 1승 0패 0무,1전 1승 0패 0무,0전 0승 0패 0무
쥬드,2전 2승 0패 0무,2전 2승 0패 0무,,3전 2승 1패 0무,3전 3승 0패 0무,4전 1승 3패 0무,2전 0승 2패 0무,4전 1승 3패 0무,1전 1승 0패 0무,2전 2승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,1전 1승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무
김연주,3전 1승 2패 0무,2전 0승 1패 1무,3전 2승 1패 0무,,1전 0승 1패 0무,2전 0승 2패 0무,2전 0승 1패 1무,1전 1승 0패 0무,1전 0승 1패 0무,2전 0승 2패 0무,1전 1승 0패 0무,2전 1승 0패 1무,0전 0승 0패 0무,1전 0승 1패 0무,1전 1승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,1전 1승 0패 0무
미지,1전 1승 0패 0무,1전 1승 0패 0무,3전 3승 0패 0무,1전 0승 1패 0무,,3전 1승 2패 0무,3전 2승 1패 0무,3전 1승 2패 0무,2전 2승 0패 0무,0전 0승 0패 0무,1전 0승 1패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무
태보,1전 0승 1패 0무,3전 2승 1패 0무,4전 1승 3패 0무,2전 0승 2패 0무,3전 1승 2패 0무,,4전 1승 3패 0무,2전 0승 2패 0무,1전 0승 1패 0무,1전 1승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,2전 1승 0패 1무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무
정현,3전 1승 1패 1무,2전 1승 0패 1무,2전 0승 2패 0무,2전 0승 1패 1무,3전 2승 1패 0무,4전 1승 3패 0무,,9전 4승 4패 1무,2전 1승 1패 0무,0전 0승 0패 0무,2전 1승 1패 0무,1전 1승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,2전 1승 0패 1무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무
파란하늘,1전 0승 1패 0무,0전 0승 0패 0무,4전 1승 3패 0무,1전 1승 0패 0무,3전 1승 2패 0무,2전 0승 2패 0무,9전 4승 4패 1무,,3전 0승 2패 1무,0전 0승 0패 0무,2전 2승 0패 0무,0전 0승 0패 0무,1전 1승 0패 0무,1전 1승 0패 0무,0전 0승 0패 0무,1전 1승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무
챨리심,1전 0승 1패 0무,0전 0승 0패 0무,1전 1승 0패 0무,1전 0승 1패 0무,2전 2승 0패 0무,1전 0승 1패 0무,2전 1승 1패 0무,3전 0승 2패 1무,,0전 0승 0패 0무,2전 0승 2패 0무,0전 0승 0패 0무,0전 0승 0패 0무,1전 0승 1패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무



Opponent Results:


Opponent,성현,예스욱,쥬드,김연주,미지,태보,정현,파란하늘,챨리심,kohei,유하,쿤,태규,똥시,민수,혜빈,테니스율,시호
Player,,,,,,,,,,,,,,,,,,
성현,,6전 0승 5패 1무,5전 1승 4패 0무,3전 0승 1패 2무,1전 1승 0패 0무,4전 2승 2패 0무,2전 1승 1패 0무,5전 2승 3패 0무,2전 2승 0패 0무,0전 0승 0패 0무,3전 1승 2패 0무,2전 0승 1패 1무,2전 1승 1패 0무,0전 0승 0패 0무,1전 1승 0패 0무,1전 0승 1패 0무,1전 0승 1패 0무,0전 0승 0패 0무
예스욱,6전 5승 0패 1무,,5전 4승 1패 0무,0전 0승 0패 0무,1전 0승 1패 0무,4전 1승 2패 1무,7전 4승 1패 2무,3전 3승 0패 0무,1전 1승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,1전 1승 0패 0무,1전 1승 0패 0무,0전 0승 0패 0무,5전 2승 1패 2무,1전 1승 0패 0무,1전 1승 0패 0무,0전 0승 0패 0무
쥬드,5전 4승 1패 0무,5전 1승 4패 0무,,0전 0승 0패 0무,4전 1승 3패 0무,7전 6승 1패 0무,13전 7승 6패 0무,2전 2승 0패 0무,5전 4승 1패 0무,0전 0승 0패 0무,3전 3승 0패 0무,3전 1승 2패 0무,0전 0승 0패 0무,0전 0승 0패 0무,1전 1승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무
김연주,3전 1승 0패 2무,0전 0승 0패 0무,0전 0승 0패 0무,,8전 4승 4패 0무,3전 1승 2패 0무,8전 1승 6패 1무,10전 3승 6패 1무,4전 2승 1패 1무,0전 0승 0패 0무,8전 4승 3패 1무,1전 0승 1패 0무,0전 0승 0패 0무,1전 0승 1패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무
미지,1전 0승 1패 0무,1전 1승 0패 0무,4전 3승 1패 0무,8전 4승 4패 0무,,2전 2승 0패 0무,5전 4승 1패 0무,8전 5승 3패 0무,1전 1승 0패 0무,1전 1승 0패 0무,3전 1승 2패 0무,1전 0승 1패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,1전 0승 1패 0무
태보,4전 2승 2패 0무,4전 2승 1패 1무,7전 1승 6패 0무,3전 2승 1패 0무,2전 0승 2패 0무,,8전 1승 6패 1무,4전 1승 3패 0무,5전 3승 2패 0무,2전 1승 1패 0무,1전 0승 1패 0무,2전 0승 2패 0무,0전 0승 0패 0무,2전 1승 1패 0무,2전 0승 2패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무
정현,2전 1승 1패 0무,7전 1승 4패 2무,13전 6승 7패 0무,8전 6승 1패 1무,5전 1승 4패 0무,8전 6승 1패 1무,,4전 2승 1패 1무,4전 1승 2패 1무,2전 1승 1패 0무,3전 0승 2패 1무,1전 0승 1패 0무,2전 0승 0패 2무,1전 1승 0패 0무,4전 0승 3패 1무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무
파란하늘,5전 3승 2패 0무,3전 0승 3패 0무,2전 0승 2패 0무,10전 6승 3패 1무,8전 3승 5패 0무,4전 3승 1패 0무,4전 1승 2패 1무,,2전 1승 1패 0무,2전 0승 2패 0무,7전 3승 3패 1무,0전 0승 0패 0무,1전 0승 0패 1무,1전 1승 0패 0무,2전 0승 2패 0무,1전 1승 0패 0무,3전 2승 1패 0무,1전 0승 1패 0무
챨리심,2전 0승 2패 0무,1전 0승 1패 0무,5전 1승 4패 0무,4전 1승 2패 1무,1전 0승 1패 0무,5전 2승 3패 0무,4전 2승 1패 1무,2전 1승 1패 0무,,3전 1승 2패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무,1전 0승 1패 0무,0전 0승 0패 0무,0전 0승 0패 0무,0전 0승 0패 0무


In [30]:
########################### FAIL ##############################################

import pandas as pd

# Assuming players_df and matches_df have already been loaded with your actual data
# For demonstration purposes, let's assume some sample data
# players_df = pd.read_excel('path_to_your_players_sheet.xlsx')
# matches_df = pd.read_excel('path_to_your_matches_sheet.xlsx')

# Get player nicknames from players DataFrame
players = players_df['닉네임'].values  # Get player nicknames from players DataFrame
matches = matches_df  # Load matches DataFrame

# Initialize DataFrames for pairwise results
same_team_results = pd.DataFrame(columns=players, index=players).fillna('0전 0승 0패 0무')
opponent_results = pd.DataFrame(columns=players, index=players).fillna('0전 0승 0패 0무')

# Function to update results
def update_results(df, player1, player2, result):
    current = df.at[player1, player2]
    
    if current == '0전 0승 0패 0무':
        current_matches, current_wins, current_losses, current_draws = 0, 0, 0, 0
    else:
        current_matches, current_wins, current_losses, current_draws = map(int, [
            current.split('전')[0],
            current.split(' ')[1].replace('승', ''),
            current.split(' ')[2].replace('패', ''),
            current.split(' ')[3].replace('무', '')
        ])
    
    current_matches += 1  # Increment the match count
    if result == 'win':
        current_wins += 1
    elif result == 'loss':
        current_losses += 1
    else:  # result == 'draw'
        current_draws += 1
    
    df.at[player1, player2] = f"{current_matches}전 {current_wins}승 {current_losses}패 {current_draws}무"

# Fill the DataFrames with results
for index, row in matches.iterrows():
    team1 = [row['team1_forehand'], row['team1_backhand']]
    team2 = [row['team2_forehand'], row['team2_backhand']]
    
    # Determine match result
    if row['team1_score'] > row['team_2_score']:
        result_team1 = 'win'
        result_team2 = 'loss'
    elif row['team1_score'] < row['team_2_score']:
        result_team1 = 'loss'
        result_team2 = 'win'
    else:
        result_team1 = 'draw'
        result_team2 = 'draw'

    # Update same team results
    for player1 in team1:
        for player2 in team1:
            if player1 != player2:
                update_results(same_team_results, player1, player2, result_team1)
            else:
                same_team_results.at[player1, player2] = ''  # Leave empty if same player

    for player1 in team2:
        for player2 in team2:
            if player1 != player2:
                update_results(same_team_results, player1, player2, result_team2)
            else:
                same_team_results.at[player1, player2] = ''  # Leave empty if same player

    # Update opponent results
    for player1 in team1:
        for player2 in team2:
            update_results(opponent_results, player1, player2, result_team1)
            update_results(opponent_results, player2, player1, result_team2)

# Clean the outputs to avoid multiple entries for the same match
same_team_results = same_team_results.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
opponent_results = opponent_results.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Calculate winning percentages
def calculate_win_percentage(df):
    win_percentage = {}
    for player in df.index:
        matches_played = 0
        wins = 0
        for teammate in df.columns:
            if player != teammate:
                current = df.at[player, teammate]
                if current != '0전 0승 0패 0무':
                    matches_played += int(current.split('전')[0])
                    wins += int(current.split(' ')[1].replace('승', ''))
        win_percentage[player] = round((wins / matches_played) * 100, 1) if matches_played > 0 else 0

    return win_percentage

# Add win percentages to DataFrames
same_team_win_percentage = calculate_win_percentage(same_team_results)
opponent_win_percentage = calculate_win_percentage(opponent_results)

# Format the results to include winning percentages
def format_results_with_win_percentage(df, win_percentage):
    for player in df.index:
        for teammate in df.columns:
            if player != teammate and df.at[player, teammate] != '':
                df.at[player, teammate] += f" ({win_percentage[player]}%)"
    return df

same_team_results = format_results_with_win_percentage(same_team_results, same_team_win_percentage)
opponent_results = format_results_with_win_percentage(opponent_results, opponent_win_percentage)

# Set index and column names for clarity
same_team_results.index.name = 'Player'
same_team_results.columns.name = 'Teammate'

opponent_results.index.name = 'Player'
opponent_results.columns.name = 'Opponent'

# Display the results at the end
print("Same Team Results:")
display(same_team_results)

print("\nOpponent Results:")
display(opponent_results)


Same Team Results:


Teammate,성현,예스욱,쥬드,김연주,미지,태보,정현,파란하늘,챨리심,kohei,유하,쿤,태규,똥시,민수,혜빈,테니스율,시호
Player,,,,,,,,,,,,,,,,,,
성현,,2전 1승 1패 0무 (31.6%),2전 2승 0패 0무 (31.6%),3전 1승 2패 0무 (31.6%),1전 1승 0패 0무 (31.6%),1전 0승 1패 0무 (31.6%),3전 1승 1패 1무 (31.6%),1전 0승 1패 0무 (31.6%),1전 0승 1패 0무 (31.6%),0전 0승 0패 0무 (31.6%),3전 0승 2패 1무 (31.6%),0전 0승 0패 0무 (31.6%),0전 0승 0패 0무 (31.6%),0전 0승 0패 0무 (31.6%),0전 0승 0패 0무 (31.6%),1전 0승 1패 0무 (31.6%),1전 0승 1패 0무 (31.6%),0전 0승 0패 0무 (31.6%)
예스욱,2전 1승 1패 0무 (66.7%),,2전 2승 0패 0무 (66.7%),2전 0승 1패 1무 (66.7%),1전 1승 0패 0무 (66.7%),3전 2승 1패 0무 (66.7%),2전 1승 0패 1무 (66.7%),0전 0승 0패 0무 (66.7%),0전 0승 0패 0무 (66.7%),0전 0승 0패 0무 (66.7%),2전 2승 0패 0무 (66.7%),1전 1승 0패 0무 (66.7%),1전 0승 0패 1무 (66.7%),0전 0승 0패 0무 (66.7%),0전 0승 0패 0무 (66.7%),1전 1승 0패 0무 (66.7%),1전 1승 0패 0무 (66.7%),0전 0승 0패 0무 (66.7%)
쥬드,2전 2승 0패 0무 (62.5%),2전 2승 0패 0무 (62.5%),,3전 2승 1패 0무 (62.5%),3전 3승 0패 0무 (62.5%),4전 1승 3패 0무 (62.5%),2전 0승 2패 0무 (62.5%),4전 1승 3패 0무 (62.5%),1전 1승 0패 0무 (62.5%),2전 2승 0패 0무 (62.5%),0전 0승 0패 0무 (62.5%),0전 0승 0패 0무 (62.5%),0전 0승 0패 0무 (62.5%),0전 0승 0패 0무 (62.5%),1전 1승 0패 0무 (62.5%),0전 0승 0패 0무 (62.5%),0전 0승 0패 0무 (62.5%),0전 0승 0패 0무 (62.5%)
김연주,3전 1승 2패 0무 (34.8%),2전 0승 1패 1무 (34.8%),3전 2승 1패 0무 (34.8%),,1전 0승 1패 0무 (34.8%),2전 0승 2패 0무 (34.8%),2전 0승 1패 1무 (34.8%),1전 1승 0패 0무 (34.8%),1전 0승 1패 0무 (34.8%),2전 0승 2패 0무 (34.8%),1전 1승 0패 0무 (34.8%),2전 1승 0패 1무 (34.8%),0전 0승 0패 0무 (34.8%),1전 0승 1패 0무 (34.8%),1전 1승 0패 0무 (34.8%),0전 0승 0패 0무 (34.8%),0전 0승 0패 0무 (34.8%),1전 1승 0패 0무 (34.8%)
미지,1전 1승 0패 0무 (61.1%),1전 1승 0패 0무 (61.1%),3전 3승 0패 0무 (61.1%),1전 0승 1패 0무 (61.1%),,3전 1승 2패 0무 (61.1%),3전 2승 1패 0무 (61.1%),3전 1승 2패 0무 (61.1%),2전 2승 0패 0무 (61.1%),0전 0승 0패 0무 (61.1%),1전 0승 1패 0무 (61.1%),0전 0승 0패 0무 (61.1%),0전 0승 0패 0무 (61.1%),0전 0승 0패 0무 (61.1%),0전 0승 0패 0무 (61.1%),0전 0승 0패 0무 (61.1%),0전 0승 0패 0무 (61.1%),0전 0승 0패 0무 (61.1%)
태보,1전 0승 1패 0무 (30.4%),3전 2승 1패 0무 (30.4%),4전 1승 3패 0무 (30.4%),2전 0승 2패 0무 (30.4%),3전 1승 2패 0무 (30.4%),,4전 1승 3패 0무 (30.4%),2전 0승 2패 0무 (30.4%),1전 0승 1패 0무 (30.4%),1전 1승 0패 0무 (30.4%),0전 0승 0패 0무 (30.4%),0전 0승 0패 0무 (30.4%),0전 0승 0패 0무 (30.4%),0전 0승 0패 0무 (30.4%),2전 1승 0패 1무 (30.4%),0전 0승 0패 0무 (30.4%),0전 0승 0패 0무 (30.4%),0전 0승 0패 0무 (30.4%)
정현,3전 1승 1패 1무 (40.6%),2전 1승 0패 1무 (40.6%),2전 0승 2패 0무 (40.6%),2전 0승 1패 1무 (40.6%),3전 2승 1패 0무 (40.6%),4전 1승 3패 0무 (40.6%),,9전 4승 4패 1무 (40.6%),2전 1승 1패 0무 (40.6%),0전 0승 0패 0무 (40.6%),2전 1승 1패 0무 (40.6%),1전 1승 0패 0무 (40.6%),0전 0승 0패 0무 (40.6%),0전 0승 0패 0무 (40.6%),2전 1승 0패 1무 (40.6%),0전 0승 0패 0무 (40.6%),0전 0승 0패 0무 (40.6%),0전 0승 0패 0무 (40.6%)
파란하늘,1전 0승 1패 0무 (42.9%),0전 0승 0패 0무 (42.9%),4전 1승 3패 0무 (42.9%),1전 1승 0패 0무 (42.9%),3전 1승 2패 0무 (42.9%),2전 0승 2패 0무 (42.9%),9전 4승 4패 1무 (42.9%),,3전 0승 2패 1무 (42.9%),0전 0승 0패 0무 (42.9%),2전 2승 0패 0무 (42.9%),0전 0승 0패 0무 (42.9%),1전 1승 0패 0무 (42.9%),1전 1승 0패 0무 (42.9%),0전 0승 0패 0무 (42.9%),1전 1승 0패 0무 (42.9%),0전 0승 0패 0무 (42.9%),0전 0승 0패 0무 (42.9%)
챨리심,1전 0승 1패 0무 (28.6%),0전 0승 0패 0무 (28.6%),1전 1승 0패 0무 (28.6%),1전 0승 1패 0무 (28.6%),2전 2승 0패 0무 (28.6%),1전 0승 1패 0무 (28.6%),2전 1승 1패 0무 (28.6%),3전 0승 2패 1무 (28.6%),,0전 0승 0패 0무 (28.6%),2전 0승 2패 0무 (28.6%),0전 0승 0패 0무 (28.6%),0전 0승 0패 0무 (28.6%),1전 0승 1패 0무 (28.6%),0전 0승 0패 0무 (28.6%),0전 0승 0패 0무 (28.6%),0전 0승 0패 0무 (28.6%),0전 0승 0패 0무 (28.6%)



Opponent Results:


Opponent,성현,예스욱,쥬드,김연주,미지,태보,정현,파란하늘,챨리심,kohei,유하,쿤,태규,똥시,민수,혜빈,테니스율,시호
Player,,,,,,,,,,,,,,,,,,
성현,0전 0승 0패 0무,6전 0승 5패 1무 (31.6%),5전 1승 4패 0무 (31.6%),3전 0승 1패 2무 (31.6%),1전 1승 0패 0무 (31.6%),4전 2승 2패 0무 (31.6%),2전 1승 1패 0무 (31.6%),5전 2승 3패 0무 (31.6%),2전 2승 0패 0무 (31.6%),0전 0승 0패 0무 (31.6%),3전 1승 2패 0무 (31.6%),2전 0승 1패 1무 (31.6%),2전 1승 1패 0무 (31.6%),0전 0승 0패 0무 (31.6%),1전 1승 0패 0무 (31.6%),1전 0승 1패 0무 (31.6%),1전 0승 1패 0무 (31.6%),0전 0승 0패 0무 (31.6%)
예스욱,6전 5승 0패 1무 (66.7%),0전 0승 0패 0무,5전 4승 1패 0무 (66.7%),0전 0승 0패 0무 (66.7%),1전 0승 1패 0무 (66.7%),4전 1승 2패 1무 (66.7%),7전 4승 1패 2무 (66.7%),3전 3승 0패 0무 (66.7%),1전 1승 0패 0무 (66.7%),0전 0승 0패 0무 (66.7%),0전 0승 0패 0무 (66.7%),1전 1승 0패 0무 (66.7%),1전 1승 0패 0무 (66.7%),0전 0승 0패 0무 (66.7%),5전 2승 1패 2무 (66.7%),1전 1승 0패 0무 (66.7%),1전 1승 0패 0무 (66.7%),0전 0승 0패 0무 (66.7%)
쥬드,5전 4승 1패 0무 (62.5%),5전 1승 4패 0무 (62.5%),0전 0승 0패 0무,0전 0승 0패 0무 (62.5%),4전 1승 3패 0무 (62.5%),7전 6승 1패 0무 (62.5%),13전 7승 6패 0무 (62.5%),2전 2승 0패 0무 (62.5%),5전 4승 1패 0무 (62.5%),0전 0승 0패 0무 (62.5%),3전 3승 0패 0무 (62.5%),3전 1승 2패 0무 (62.5%),0전 0승 0패 0무 (62.5%),0전 0승 0패 0무 (62.5%),1전 1승 0패 0무 (62.5%),0전 0승 0패 0무 (62.5%),0전 0승 0패 0무 (62.5%),0전 0승 0패 0무 (62.5%)
김연주,3전 1승 0패 2무 (34.8%),0전 0승 0패 0무 (34.8%),0전 0승 0패 0무 (34.8%),0전 0승 0패 0무,8전 4승 4패 0무 (34.8%),3전 1승 2패 0무 (34.8%),8전 1승 6패 1무 (34.8%),10전 3승 6패 1무 (34.8%),4전 2승 1패 1무 (34.8%),0전 0승 0패 0무 (34.8%),8전 4승 3패 1무 (34.8%),1전 0승 1패 0무 (34.8%),0전 0승 0패 0무 (34.8%),1전 0승 1패 0무 (34.8%),0전 0승 0패 0무 (34.8%),0전 0승 0패 0무 (34.8%),0전 0승 0패 0무 (34.8%),0전 0승 0패 0무 (34.8%)
미지,1전 0승 1패 0무 (61.1%),1전 1승 0패 0무 (61.1%),4전 3승 1패 0무 (61.1%),8전 4승 4패 0무 (61.1%),0전 0승 0패 0무,2전 2승 0패 0무 (61.1%),5전 4승 1패 0무 (61.1%),8전 5승 3패 0무 (61.1%),1전 1승 0패 0무 (61.1%),1전 1승 0패 0무 (61.1%),3전 1승 2패 0무 (61.1%),1전 0승 1패 0무 (61.1%),0전 0승 0패 0무 (61.1%),0전 0승 0패 0무 (61.1%),0전 0승 0패 0무 (61.1%),0전 0승 0패 0무 (61.1%),0전 0승 0패 0무 (61.1%),1전 0승 1패 0무 (61.1%)
태보,4전 2승 2패 0무 (30.4%),4전 2승 1패 1무 (30.4%),7전 1승 6패 0무 (30.4%),3전 2승 1패 0무 (30.4%),2전 0승 2패 0무 (30.4%),0전 0승 0패 0무,8전 1승 6패 1무 (30.4%),4전 1승 3패 0무 (30.4%),5전 3승 2패 0무 (30.4%),2전 1승 1패 0무 (30.4%),1전 0승 1패 0무 (30.4%),2전 0승 2패 0무 (30.4%),0전 0승 0패 0무 (30.4%),2전 1승 1패 0무 (30.4%),2전 0승 2패 0무 (30.4%),0전 0승 0패 0무 (30.4%),0전 0승 0패 0무 (30.4%),0전 0승 0패 0무 (30.4%)
정현,2전 1승 1패 0무 (40.6%),7전 1승 4패 2무 (40.6%),13전 6승 7패 0무 (40.6%),8전 6승 1패 1무 (40.6%),5전 1승 4패 0무 (40.6%),8전 6승 1패 1무 (40.6%),0전 0승 0패 0무,4전 2승 1패 1무 (40.6%),4전 1승 2패 1무 (40.6%),2전 1승 1패 0무 (40.6%),3전 0승 2패 1무 (40.6%),1전 0승 1패 0무 (40.6%),2전 0승 0패 2무 (40.6%),1전 1승 0패 0무 (40.6%),4전 0승 3패 1무 (40.6%),0전 0승 0패 0무 (40.6%),0전 0승 0패 0무 (40.6%),0전 0승 0패 0무 (40.6%)
파란하늘,5전 3승 2패 0무 (42.9%),3전 0승 3패 0무 (42.9%),2전 0승 2패 0무 (42.9%),10전 6승 3패 1무 (42.9%),8전 3승 5패 0무 (42.9%),4전 3승 1패 0무 (42.9%),4전 1승 2패 1무 (42.9%),0전 0승 0패 0무,2전 1승 1패 0무 (42.9%),2전 0승 2패 0무 (42.9%),7전 3승 3패 1무 (42.9%),0전 0승 0패 0무 (42.9%),1전 0승 0패 1무 (42.9%),1전 1승 0패 0무 (42.9%),2전 0승 2패 0무 (42.9%),1전 1승 0패 0무 (42.9%),3전 2승 1패 0무 (42.9%),1전 0승 1패 0무 (42.9%)
챨리심,2전 0승 2패 0무 (28.6%),1전 0승 1패 0무 (28.6%),5전 1승 4패 0무 (28.6%),4전 1승 2패 1무 (28.6%),1전 0승 1패 0무 (28.6%),5전 2승 3패 0무 (28.6%),4전 2승 1패 1무 (28.6%),2전 1승 1패 0무 (28.6%),0전 0승 0패 0무,3전 1승 2패 0무 (28.6%),0전 0승 0패 0무 (28.6%),0전 0승 0패 0무 (28.6%),0전 0승 0패 0무 (28.6%),0전 0승 0패 0무 (28.6%),1전 0승 1패 0무 (28.6%),0전 0승 0패 0무 (28.6%),0전 0승 0패 0무 (28.6%),0전 0승 0패 0무 (28.6%)
